In [ ]:
# Import the needed libraries and modueles for the data cleaning effort
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Downloaded a CSV file from Kaggle.com Title Top 10 Spotify and YouTube
# Statistics for the Top 10 songs of various Spotify artists and their yt video.

songs_base_df = pd.read_csv("./Resources/Spotify_Youtube.csv")
songs_base_df.head()

In [ ]:
# The CSV file had and index so need to remove that column
songs_cleaned_df = songs_base_df.drop(columns='Unnamed: 0')
print(songs_cleaned_df.columns)
songs_cleaned_df.head()

In [ ]:
#Need to see the different data types of the columns
print(songs_cleaned_df.dtypes)

In [ ]:
#This is removing any object column that dosn't help with plots
songs_reduced_df = songs_base_df.drop(columns=['Unnamed: 0', 'Url_spotify', 'Album_type', 'Uri', 'Url_youtube', 
                                     'Title', 'Channel', 'Description', 'Licensed', 'official_video'])
print(songs_reduced_df.dtypes)
songs_reduced_df.head()

In [ ]:
# Decision point 
# We decided to reduce that dataset to nothing with an NaN.  We looked for which 
# columns had NaN and decide to drop all with NaN
songs_reduced_df.isna().any()

In [ ]:
#Looking for NaNs
songs_reduced_df.count()

In [ ]:
# Which rows are going to get removed
songs_reduced_df[songs_reduced_df.isna().any(axis=1)]  

In [ ]:
# Removed all columns with NaN 
# This became our Baseline Data Frame --no long a top 10 per artest data frame
songs_reduced_noNaN = songs_reduced_df.dropna(how='any') #['Likes']
songs_reduced_noNaN.count()

In [ ]:
# From the noNaN data frame we sorted by 'Likes' and created a data frame of the top 100
top_100_liked_df = songs_reduced_noNaN.sort_values('Likes',ascending=False)
top_100_liked_df = top_100_liked_df.head(100)
top_100_liked_df

In [ ]:
# We wanted to see how many of the top 100 liked songs were from the same artist
top_100_liked_df.value_counts(subset='Artist')

In [ ]:
# Not all are divided by 10 with using the _noNaN data frame but has correct ratio
# This is a little explorative code if we were going to expand to all of the artist and what their average likes were 
# creating the beginning of a new data frame instead of using the top 100.  We went with top 100
mean_artist_likes = songs_reduced_noNaN.groupby(["Artist"])["Likes"].agg('mean')
mean_artist_likes

In [ ]:
# Created a data frame from the above aggregation
artist_likes_summary = pd.DataFrame(
                        {"Average Likes": mean_artist_likes,
                         })
artist_likes_summary.sort_values('Average Likes', ascending=False)

In [ ]:
# This is pulled from the top 100 tracks some artist had more than one track on the top 100 likes
# This is a the summary table we decided to use for our scatter plots
per_artist_summary = top_100_liked_df.groupby(['Artist']).agg({'Likes':'mean','Valence':'mean','Speechiness':'mean',
                                                           'Acousticness':'mean','Instrumentalness':'mean','Liveness':'mean',
                                                           'Energy':'mean'})
per_artist_summary_df = per_artist_summary.sort_values('Likes',ascending=False)
per_artist_summary_df = per_artist_summary_df.reset_index()
per_artist_summary_df

In [ ]:
# We have a need to create a second data set we can create some line charts with, but first 
# we want to reduce it to the top likes song per artist, so we removed duplicate artist keep the one with the most likes
# and then removed duplicate track meaning the only artist that got credit per track
top_artist_track = songs_reduced_noNaN.sort_values('Likes',ascending=False).drop_duplicates(subset=['Artist'],keep='first')
top_artist_track = top_artist_track.drop_duplicates(subset=['Track'],keep='first')
top_artist_track.reset_index(inplace=True)
top_artist_track

In [ ]:
# We then saved both of the wanted data frames into a CSV file and saved them to another directory
# This is the end of the data clean up and creation of data frame for use in the analysis notebook
top_artist_track.to_csv("Resources/ArtistTopTrack.csv", index=False, header=True)
per_artist_summary_df.to_csv("Resources/PerArtistSummary.csv", index=False, header=True)